## **ResNet: Training a Classifier**

## What is CIFAR10?


The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class.

Link: https://www.cs.toronto.edu/~kriz/cifar.html


# **Mission**

In the lecture, you have seen how to define neural networks, compute loss and make updates to the weights of the network. In this task, you will classify CIFAR10 images by using a ResNet.

We will do the following steps in order:

1. Load and normalize the CIFAR10 training and test datasets using torchvision
2. Define a Convolutional Neural Network
3. Define a loss function
4. Train the network on the training data
5. Test the network on the test data

# **Complete the code for each section and train the model.**

You can set the number of layers freely when constructing a model. Share the results among your team members.

In [ ]:
# import
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Hyper-parameters
num_epochs = 80
learning_rate = 0.001

In [ ]:
# Image preprocessing modules
transform = transforms.Compose([
    ############################################################################
    # Complete the code                                                        #
    ############################################################################                             
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()
    ])

In [ ]:
# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform=transforms.ToTensor())


Extracting ../../data/cifar-10-python.tar.gz to ../../data/


In [ ]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

In [ ]:
# 3x3 convolution

def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
    ############################################################################
    # Complete the code                                                        #
    ############################################################################ 
    

In [ ]:
# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        self.stride = stride
            ############################################################################
            # Complete the code                                                        #
            ############################################################################ 

    def forward(self, x):
            ############################################################################
            # Complete the code                                                        #
            ############################################################################ 
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
      
        if self.downsample is not None:
          identity = self.downsample(x)

        out += identity
        out = self.relu(out)
        
        return out


In [ ]:
# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()

        self.in_channels = 16
        self.conv = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

            ############################################################################
            # Complete the code                                                        #
            ############################################################################ 

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels:
          downsample = nn.Sequential(
              conv3x3(self.in_channels, out_channels, stride=stride),
              nn.BatchNorm2d(out_channels)
          )
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
          layers.append(block(out_channels, out_channels))
            ############################################################################
            # Complete the code                                                        #
            ############################################################################ 
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
            ############################################################################
            # Complete the code                                                        #
            ############################################################################ 
        return out

model = ResNet(ResidualBlock, [2, 2, 2]).to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [12]:
# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
            ############################################################################
            # Complete the code                                                        #
            ############################################################################ 

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            ############################################################################
            # Complete the code                                                        #
            ############################################################################ 
            
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)


Epoch [1/80], Step [100/500] Loss: 1.5190
Epoch [1/80], Step [200/500] Loss: 1.4203
Epoch [1/80], Step [300/500] Loss: 1.3825
Epoch [1/80], Step [400/500] Loss: 1.2183
Epoch [1/80], Step [500/500] Loss: 1.1009
Epoch [2/80], Step [100/500] Loss: 1.1335
Epoch [2/80], Step [200/500] Loss: 1.1554
Epoch [2/80], Step [300/500] Loss: 0.8729
Epoch [2/80], Step [400/500] Loss: 1.0143
Epoch [2/80], Step [500/500] Loss: 1.0701
Epoch [3/80], Step [100/500] Loss: 0.8709
Epoch [3/80], Step [200/500] Loss: 0.9556
Epoch [3/80], Step [300/500] Loss: 0.8561
Epoch [3/80], Step [400/500] Loss: 0.8931
Epoch [3/80], Step [500/500] Loss: 0.6584
Epoch [4/80], Step [100/500] Loss: 0.7997
Epoch [4/80], Step [200/500] Loss: 0.7263
Epoch [4/80], Step [300/500] Loss: 0.7449
Epoch [4/80], Step [400/500] Loss: 0.7438
Epoch [4/80], Step [500/500] Loss: 0.7651
Epoch [5/80], Step [100/500] Loss: 0.7790
Epoch [5/80], Step [200/500] Loss: 0.6536
Epoch [5/80], Step [300/500] Loss: 0.5827
Epoch [5/80], Step [400/500] Loss:

In [13]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 88.05 %


In [14]:
# Save the model checkpoint
torch.save(model.state_dict(), 'resnet.ckpt')